In [35]:
from constraint import *
from itertools import *

# Configuration and Setup

In [36]:
# Configure day naming constants
M,T,W,TH,F = ['monday','tuesday','wednesday','thursday','friday']

In [37]:
# Configure number of free hours each day
#
# Sample :
#   free_time[W] = 3  # Sets wednesday to 3 free hours

free_time = {}
free_time[M]  = 2  # Monday
free_time[T]  = 1  # Tuesday
free_time[W]  = 1  # Wednesday
free_time[TH] = 1  # Thursday
free_time[F]  = 1  # Friday

In [38]:
# Configure tasks to complete this week
#
# Sample tasks
#   tasks['name'] = { 'effort': 1 }

tasks = {}
tasks['laundry'] = { 'effort': 2 }
tasks['pay bills'] = { 'effort': 2 }
tasks['wash the car'] = { 'effort': 2 }

# Precalculate intermediate values

In [39]:
# Calculate helper variables
# Descriptions below
total_free_hours = 0
hour_to_day = {}

for day in [M,T,W,TH,F]:
    for hour in range(0,free_time[day]):
      hour_to_day[total_free_hours] = day
      total_free_hours += 1

hours_in_week = range(0,total_free_hours)

In [40]:
# hour_to_day is a mapping of hour in week to a day
hour_to_day

{0: 'monday',
 1: 'monday',
 2: 'tuesday',
 3: 'wednesday',
 4: 'thursday',
 5: 'friday'}

In [41]:
# total_free_hours is a sum of all the free hours this week 
total_free_hours

6

In [42]:
# hours_in_week is an incremental array of hours in a week
hours_in_week

[0, 1, 2, 3, 4, 5]

# Defining the constraint solver

In [28]:
problem = Problem()
# task.keys() , i.e. ['laundry', 'pay bills', ...] are the variables
# hours_in_week, i.e. [0,1,2,3,...] are the possible values for the variables
# 
# The constraint solver will try assigning different hours of the week
# to the variables, and reject all those that do not meet the constraints
problem.addVariables(tasks.keys(),hours_in_week)

## Add Constraint: Two tasks cannot begin at the same time

In [29]:
# Add a constraint so that no task begins on an identical hour to another task
problem.addConstraint(AllDifferentConstraint())

In [ ]:
Get a possible 

In [30]:
# Get a solution
solution = problem.getSolution()
solution

{'laundry': 5, 'pay bills': 4, 'wash the car': 3}